# Работа с базами данных в Python

## DB API
DB API это стандартизированный набор методов для работы с базами данных в Python. 

![db-api](dbapi.png)

Предварительная подготовка: 
1. Запустить БД, если она еще не запущена. Будем использовать mssql в docker `docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=1234qwE?" -e "MSSQL_PID=Express" -p 1433:1433 -d mcr.microsoft.com/mssql/server:2022-latest`
2. Установить драйвер для mssql `uv add mssql-python`
3. Научимся подключаться из приложений для работы с бд (dbeaver, datagrip и тп)
4. Создать тестовую базу данных и наполнить ее данными

In [ ]:
!docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=1234qwE?" -e "MSSQL_PID=Express" -p 1433:1433 -d mcr.microsoft.com/mssql/server:2022-latest

In [ ]:
!uv add mssql-python
!pip install mssql-python

Resolved 47 packages in 15ms
Audited 45 packages in 9ms


# Установка соединения

In [1]:
# Просто подключимся к БД

import mssql_python
connection_string = 'SERVER=localhost;PORT=1433;DATABASE=todo;UID=sa;PWD=1234qwE?;Encrypt=no'
conn = mssql_python.connect(connection_string)
cur = conn.cursor()

# TODO

cur.close()
conn.close()

In [3]:
# Просто подключимся к БД

import mssql_python
connection_string = 'SERVER=localhost;PORT=1433;DATABASE=todo;UID=sa;PWD=1234qwE?;Encrypt=no'
# Для Windows
# connection_string = 'SERVER=.;DATABASE=todo;Authentication=ActiveDirectoryIntegrated;Encrypt=no'
with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        pass
        # TODO


## Что тут происходит
1. Создали connection string
2. Открыли подключение, передав строку в `connect`
3. Создали курсор - объект для работы с БД
4. Затем, так как подключение и курсор это внешние по отношению к питону ресурсы мы должны их освободить с помощью методов `close`

## Как устроен connection string
Строка это набор элементов `<key>=<value>` описывающая то как надо подключаться к БД

|key|Описание|
|:--|:-------|
|Server|Адрес подключения к бд|
|Port|Порт подключения к бд|
|Database|Имя самой базы данных|
|UID|Имя пользователя|
|PWD|Пароль пользователя|
|Encrypt|Включение/выключение шифрования отправляемых данных|
|Authentication|Способ аутентификации пользователя|

Кроме этих полей есть еще несколько других о которых можно прочитать в [документации](https://github.com/microsoft/mssql-python/wiki/Connection-to-SQL-Database#connection-string) к драйверу.

# Чтение данных из БД

In [4]:
# Читаем из БД

with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute("select * from items")

        print(cur.fetchall())

[('buy milk', False), ('do homework', True), ('read book', True), ('play games', False)]


Выгружать из базы все не всегда удобно (хотя курсор все равно выгржает все данные в память), поэтому иногда имеет смысл читать ответ построчно

In [ ]:
# Читаем из БД построчно
with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute("select * from items where done=1")
        
        print(cur.fetchone())
        print(cur.fetchone())
        print(cur.fetchone())
        print(cur.fetchone())
        print(cur.fetchone())
        print(cur.fetchone())


('do homework', True)
('read book', True)
None
None
None
None


In [ ]:
# Курсор как итератор
with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute("select * from items where done=1")
        
        for row in cur:
            print(row)

('do homework', True)
('read book', True)


# Запись данных в БД

In [15]:
# Сперва пишем, а потом читаем
with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute("insert into items values('watch movie', 0)")
        cur.commit() # сохраняем данные в таблицу
        cur.execute("select * from items")
        for row in cur:
            print(row)


('buy milk', False)
('do homework', True)
('read book', True)
('play games', False)
('watch movie', False)
('watch movie', False)


In [16]:
with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        cur.execute("select * from items")
        for row in cur:
            print(row)

('buy milk', False)
('do homework', True)
('read book', True)
('play games', False)
('watch movie', False)
('watch movie', False)


Часто бывает необходимо выполнять параметризованные запросы: пользователь передает в вашу функцию название айтема и флаг выполнен ли он, а вы должны вставить. Ваши предложения, как это можно сделать?

In [ ]:
def insert_item(cur: mssql_python.Cursor, name: str, done: bool):
    cur.execute(f"insert into items values ('{name}', {1 if done else 0})")


with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        # sql injection!!!
        # insert_item(cur, "xxx', 1); drop table items; --", True)
        cur.commit() # сохраняем данные в таблицу
        cur.execute("select * from items")
        for row in cur:
            print(row)


ProgrammingError: Driver Error: Base table or view not found; DDBC Error: [Microsoft][SQL Server]Invalid object name 'items'.

In [ ]:
# Передача параметризованных запросов: именованные и не именованные параметры

def insert_item_named_args(cur: mssql_python.Cursor, item_name: str, done: bool):
    cur.execute("INSERT INTO items VALUES (:item, :done)", {
        "item": item_name,
        "done": done
    })

with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        insert_item_named_args(cur, "play games", True)
        cur.commit() # сохраняем данные в таблицу
        cur.execute("select * from items")
        for row in cur:
            print(row)

TypeError: Unsupported parameter type: The driver cannot safely convert it to a SQL type.

Что делать если хотим вставить произвольное количество строк в таблицу? 

In [31]:
# Передача множества параметров
def insert_item_named_args(cur: mssql_python.Cursor, items: list[tuple[str, bool]]):
    cur.executemany("INSERT INTO items VALUES (?, ?)", items)

with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        insert_item_named_args(cur, [("play games", True), ("make dinner", False)])
        cur.commit() # сохраняем данные в таблицу
        cur.execute("select * from items")
        for row in cur:
            print(row)

("xxx', 1); drop table items; --", True)
('watch tv', True)
('play videogames', True)
('watch tv', True)
('play games', True)
('make dinner', False)


# Транзакции

Транзакция - это последовательность операций, которая может выполниться полностью, либо не выполниться вообще. При создании подключения и курсора создается новая транзакция в рамках которой выполняются операции над БД. После этого транзакцию можно либо применить (`commit`), либо откатить (`rollback`). Если закрыть подключение без применения транзакции, то все выполненные изменения откатятся.

In [33]:
# Напишем код с коммитом транзакции

def insert_item_named_args(cur: mssql_python.Cursor, items: list[tuple[str, bool]]):
    raise EOFError()
    cur.executemany("INSERT INTO items VALUES (?, ?)", items)

with mssql_python.connect(connection_string) as conn:
    with conn.cursor() as cur:
        try:
            insert_item_named_args(cur, [("play games", True), ("make dinner", False)])
            conn.commit()
        except:
            conn.rollback()

In [34]:
import mssql_python

connection_string = 'SERVER=localhost;PORT=1433;DATABASE=todo;UID=sa;PWD=1234qwE?;Encrypt=no'
conn = mssql_python.connect(connection_string)
cur = conn.cursor()

cur.execute('SELECT * from items')

print(cur.fetchall())

cur.close()
conn.close()

[("xxx', 1); drop table items; --", True), ('watch tv', True), ('play videogames', True), ('watch tv', True), ('play games', True), ('make dinner', False), ('play games', True), ('make dinner', False)]


# Как не забывать закрыть подключение

In [10]:
# Перепишем код на менеджеры контекста with
